In [ ]:
from fmr_scripts import FMR #if running in another directory - requires PYTHONPATH or sys.path to contain directory containing fmr_scripts
#import FMR #if running in same directory
import numpy as np
import matplotlib.pyplot as plt
import glob,os
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

In [ ]:
filenames = glob.glob('*.npz')
print np.array(filenames)[np.argwhere(np.array([f.find('-fitparams') for f in filenames]) > -1).flatten()]
filename = filenames[np.argwhere(np.array([f.find('-fitparams') for f in filenames]) > -1)[0][0]]
print filename

In [ ]:
extractedparams = np.load(filename)
extractedparams.files

In [ ]:
NPeaks = [len(f) for f in extractedparams['resonantFieldsOe']]
print NPeaks

In [ ]:
#essentially a reslicing
def extract(data, field, n, whichcurves):
    return np.array([data[field][i][n] for i in whichcurves])

N = np.max(NPeaks)
freqsN = []
angsN = []
HFMRN = []
errHFMRN = []
deltaHN = []
errdeltaHN = []
phaseLorentzianN = []
errphaseLorentzianN = []
peakMagnitudeN = []
errpeakMagnitudeN = []
for n in np.arange(0,N):
    whichcurves = np.where(np.array(NPeaks) > n)[0]
    freqsN.append(extractedparams['frequenciesGHz'][whichcurves])
    angsN.append(extractedparams['anglesdeg'][whichcurves])
    HFMRN.append(extract(extractedparams,'resonantFieldsOe',n,whichcurves))
    errHFMRN.append(extract(extractedparams,'resonantFieldsErrorOe',n,whichcurves))
    deltaHN.append(extract(extractedparams,'linewidthsOe',n,whichcurves))
    errdeltaHN.append(extract(extractedparams,'linewidthsErrorOe',n,whichcurves))
    phaseLorentzianN.append(extract(extractedparams,'peakPhasesdeg',n,whichcurves))
    errphaseLorentzianN.append(extract(extractedparams,'peakPhasesErrordeg',n,whichcurves))
    peakMagnitudeN.append(extract(extractedparams,'peakMagnitude',n,whichcurves))
    errpeakMagnitudeN.append(extract(extractedparams,'peakMagnitudeError',n,whichcurves))

#### Analysis picking a single mode

#### Systematically determine which frequency points to discard (saturation magnetization? poor waveguide coupling?)

In [ ]:
#need to take into account saturation magnetization
h = 6.62607004e-34*1e7 #erg-s
muB = 9.274000999e-21 #erg/G
#mu0 = 1.00000037 #G/Oe - https://www.engineeringtoolbox.com/permeability-d_1923.html
mu0 = 1. #actually measured in G not Oe
giga = 1e9

In [ ]:
OetoAperm = 1000/(4*np.pi)

In [ ]:
def IPresfield(HFMR,factor,Hcubic,angle,Meff):
    return factor*np.sqrt((HFMR+Hcubic*np.cos(4.*angle))*(HFMR+4.*np.pi*Meff+Hcubic/4.*(3.+np.cos(4.*angle))))/giga

In [ ]:
def IPlinewidth(f,H0,alpha,ms,g100,g110):
    angle = HFMRfit[0][2]
    gmuBmu0h = HFMRfit[0][0]
    fm = gmuBmu0h*4.*np.pi*ms
    sqrt = np.sqrt((f*giga)**2+(fm/2.)**2)
    mag = np.arcsin(np.sqrt((sqrt-fm/2.)/(sqrt+fm/2.)))
    return H0 + alpha/gmuBmu0h*f*giga + g100*mag*np.cos(2.*angle)**2 + g110*mag*np.cos(2.*(angle-np.pi/4.))**2

In [ ]:
Meff = []
gip = []
Hcubic = []
phi = []

deltaH0 = []
alpha = []
ms = []
g100 = []
g110 = []

HFMRfits = []
deltaHfits = []

guessH = np.array([2*muB*mu0/h, 2.5/mu0, np.pi/4., 240.])
start = np.arange(0,len(freqsN[0])-3)

for i in start:
    freqslice = slice(i,None)
    
    HFMRfit = curve_fit(IPresfield,np.array(HFMRN[0][freqslice]),freqsN[0][freqslice],guessH)
    HFMRfits.append(HFMRfit)
    Meff.append(HFMRfit[0][-1]*4*np.pi)
    gip.append(HFMRfit[0][0]*h/(muB*mu0))
    Hcubic.append(HFMRfit[0][1]*mu0/10.)
    phi.append(np.rad2deg(HFMRfit[0][2])%360)
    
    guessdH = np.array([0., .006, HFMRfit[0][-1], 6e8/HFMRfit[0][0] ,3e8/HFMRfit[0][0]])#, HFMRfit[0][2]])
    
    deltaHfit = curve_fit(IPlinewidth,freqsN[0][freqslice],np.array(deltaHN[0][freqslice]),guessdH,bounds=(0.,np.inf))
    deltaHfits.append(deltaHfit)
    deltaH0.append(deltaHfit[0][0])    
    alpha.append(deltaHfit[0][1])
    ms.append(deltaHfit[0][2])
    g100.append(deltaHfit[0][3]*HFMRfit[0][0]/1e6)
    g110.append(deltaHfit[0][4]*HFMRfit[0][0]/1e6)

In [ ]:
%matplotlib notebook
plt.errorbar(freqsN[0],HFMRN[0],fmt='.',yerr=errHFMRN[0])
for i in start:
    if i < 8:
        plt.plot(IPresfield(HFMRN[0],*HFMRfits[i][0]),HFMRN[0],'-',label=str(i))
plt.legend()
plt.xlabel('f (GHz)')
plt.ylabel('HFMR OOP (Oe)')

In [ ]:
%matplotlib notebook
plt.errorbar(freqsN[0],deltaHN[0],fmt='.',yerr=errdeltaHN[0])
for i in start:
    HFMRfit = HFMRfits[i]
    if i < 5:
        plt.plot(freqsN[0],IPlinewidth(np.array(freqsN[0]),*deltaHfits[i][0]),'-',label=str(i))
plt.legend()
plt.xlabel('f (GHz)')
plt.ylabel(r'$\Delta$H OOP (Oe)')

In [ ]:
%matplotlib notebook
plt.plot(start,Meff,'.')
plt.xlabel('starting index of fit')
plt.ylabel('Meff')

In [ ]:
%matplotlib notebook
plt.plot(start,gip,'.')
plt.xlabel('starting index of fit')
plt.ylabel('gip')

In [ ]:
%matplotlib notebook
plt.plot(start,deltaH0,'.')
plt.xlabel('starting index of fit')
plt.ylabel('deltaH0')

In [ ]:
%matplotlib notebook
plt.plot(start,alpha,'.')
plt.xlabel('starting index of fit')
plt.ylabel('alpha')

In [ ]:
%matplotlib notebook
plt.plot(start,Hcubic,'.')
plt.xlabel('starting index of fit')
plt.ylabel('cubic anisotropy field')

In [ ]:
%matplotlib notebook
plt.plot(start,phi,'.')
plt.xlabel('starting index of fit')
plt.ylabel('sample angle')

In [ ]:
%matplotlib notebook
plt.plot(start,ms,'.')
plt.xlabel('starting index of fit')
plt.ylabel('saturation magnetization')

In [ ]:
%matplotlib notebook
plt.plot(start,g100,'.')
plt.xlabel('starting index of fit')
plt.ylabel('100 2-magnon scattering')

In [ ]:
%matplotlib notebook
plt.plot(start,g110,'.')
plt.xlabel('starting index of fit')
plt.ylabel('110 2-magnon scattering')

In [ ]:
start = 0 #defined visually from above

Meff = []
gip = []
Hcubic = []
phi = []

deltaH0 = []
alpha = []
ms = []
g100 = []
g110 = []

HFMRfits = []
deltaHfits = []
guessH = np.array([2*muB*mu0/h, 2.5/mu0, np.pi/4., 240.])

stop = np.arange(start+4,len(freqsN[0]))
for i in stop:
    freqslice = slice(start,i)
    
    HFMRfit = curve_fit(IPresfield,np.array(HFMRN[0][freqslice]),freqsN[0][freqslice],guessH)
    HFMRfits.append(HFMRfit)
    Meff.append(HFMRfit[0][-1]*4*np.pi)
    gip.append(HFMRfit[0][0]*h/(muB*mu0))
    Hcubic.append(HFMRfit[0][1]*mu0/10.)
    phi.append(np.rad2deg(HFMRfit[0][2])%360)
    
    guessdH = np.array([0., .006, HFMRfit[0][-1], 6e8/HFMRfit[0][0] ,3e8/HFMRfit[0][0]])#, HFMRfit[0][2]])
    
    deltaHfit = curve_fit(IPlinewidth,freqsN[0][freqslice],np.array(deltaHN[0][freqslice]),guessdH,bounds=(0.,np.inf))
    deltaHfits.append(deltaHfit)
    deltaH0.append(deltaHfit[0][0])    
    alpha.append(deltaHfit[0][1])
    ms.append(deltaHfit[0][2])
    g100.append(deltaHfit[0][3]*HFMRfit[0][0]/1e6)
    g110.append(deltaHfit[0][4]*HFMRfit[0][0]/1e6)

In [ ]:
%matplotlib notebook
plt.errorbar(freqsN[0],deltaHN[0],fmt='.',yerr=errdeltaHN[0])
for i in stop:
    HFMRfit = HFMRfits[i-(start+4)]
    if i > 20:
        plt.plot(freqsN[0],IPlinewidth(np.array(freqsN[0]),*deltaHfits[i-(start+4)][0]),'-',label=str(i))
plt.legend()
plt.xlabel('f (GHz)')
plt.ylabel('deltaH OOP (Oe)')

In [ ]:
%matplotlib notebook
plt.plot(stop,alpha,'.')
plt.xlabel('stopping index of fit')
plt.ylabel('alpha')

In [ ]:
stop = len(freqsN[0]) #visually defined
freqslice = slice(start,stop)
index = stop-start-4-1

HFMRfit = HFMRfits[index]
deltaHfit = deltaHfits[index]

#### Display data

In [ ]:
print 'Meff: '+str(Meff[index])+' emu/cm^3'
print 'gip: '+str(gip[index])
print 'H_cubic: '+str(Hcubic[index])+' mT'
print 'phi: '+str(phi[index])+' deg'

In [ ]:
print "deltaH_0: "+str(deltaH0[index])+' Oe'
print "alpha: "+str(alpha[index])
print ms[index]
print 'Ms: '+str(ms[index]*4.*np.pi)+' emu/cm^3'
print '100: '+str(g100[index])+' MHz'
print '110: '+str(g110[index])+' MHz'

In [ ]:
%matplotlib notebook
plt.errorbar(freqsN[0],HFMRN[0],fmt='.',yerr=errHFMRN[0])
#if start != 0 or stop != len(freqsN[0]):
#    freqslice = slice(start,stop)
plt.errorbar(freqsN[0][freqslice],HFMRN[0][freqslice],fmt='.',yerr=errHFMRN[0][freqslice])
plt.plot(IPresfield(HFMRN[0],*HFMRfit[0]),HFMRN[0],'-',label='Kittel eq.')
plt.legend()
plt.ylabel(r'H$_{FMR}$ OOP (Oe)')
plt.xlabel('Frequency (GHz)')

In [ ]:
%matplotlib notebook
plt.errorbar(freqsN[0],deltaHN[0],fmt='.',yerr=errdeltaHN[0])
#if start != 0 or stop != len(freqsN[0]):
plt.errorbar(freqsN[0][freqslice],deltaHN[0][freqslice],fmt='.',yerr=errdeltaHN[0][freqslice])
plt.plot(freqsN[0],IPlinewidth(np.array(freqsN[0]),*deltaHfit[0]),'-')
plt.ylabel('FMR Linewidth OOP (Oe)')
plt.xlabel('Frequency (GHz)')

In [ ]:
fname = filename.partition('-')[0]
outfile = fname+'-params'
np.savez(outfile, frequenciesGHz = freqsN[0][freqslice],
         resonantFieldOe = HFMRN[0][freqslice], linewidthOe = deltaHN[0][freqslice],
         resonantFieldErrorOe = errHFMRN[0][freqslice], linewidthErrorOe = errdeltaHN[0][freqslice],
         peakPhasedeg = phaseLorentzianN[0][freqslice], peakMagnitude = peakMagnitudeN[0][freqslice],
         MeffectiveOe = Meff, gilbertDamping = alpha,
         orbitalgip = gip, intrinsiclinewidthOe = deltaH0,
         cubicAnisotropymT = Hcubic, sampleAngleDeg = phi,
         MsaturationOe = ms, twoMagnon100MHz = g100, twoMagnon110MHz = g110,
         resonantFieldFitCoefficients = HFMRfit[0], linewidthFitCoefficients = deltaHfit[0],
         resonantFieldFitErrors = HFMRfit[1], linewidthFitErrors = deltaHfit[1])
#insert CRO thickness, LSMO thickness into this file??